# Installation of requirements

In [ ]:
# install requirements

!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
# install requirements

!pip install torch-geometric==2.2.0 torch-sparse==0.6.16 torch-scatter==2.1.0 -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html


In [ ]:
# Connecting the drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Making sure the working directory is the one

%cd /content/drive/MyDrive/AGILE

/content/drive/MyDrive/AGILE


In [ ]:
import os

In [ ]:
# install requirements

!pip install -r requirements.txt

# The following is the content of the file 'config_pretrain.yaml'

batch_size: 512 # batch size
warm_up: 10 # warm-up epochs
epochs: 2 # total number of epochs

load_model: pretrained_gin # resume training
eval_every_n_epochs: 1 # validation frequency
save_every_n_epochs: 5 # automatic model saving frequecy
log_every_n_steps: 50 # print training log frequency

fp16_precision: False # float precision 16 (i.e. True/False)
init_lr: 0.0005 # initial learning rate for Adam
weight_decay: 1e-5 # weight decay for Adam
gpu: cuda:0 # training GPU

model:
  num_layer: 5 # number of graph conv layers
  emb_dim: 300 # embedding dimension in graph conv layers
  feat_dim: 512 # output feature dimention
  drop_ratio: 0 # dropout ratio
  pool: mean # readout pooling (i.e., mean/max/add)

aug: node # molecule graph augmentation strategy (i.e., node/subgraph/mix)
dataset:
  num_workers: 12 # dataloader number of workers
  valid_size: 0.05 # ratio of validation data
  data_path: data/{yourowndata}.csv # path of pre-training data

loss:
  temperature: 0.1 # temperature of NT-Xent loss
  use_cosine_similarity: True # whether to use cosine similarity in NT-Xent loss (i.e. True/False)


# Further ahead from here, I will try to pre-train the MolCLR model with the file 'pretrain.py'

In [8]:
import os
import shutil
import sys
import torch
import yaml
import numpy as np
from datetime import datetime

In [11]:
import torch.nn.functional as F

In [15]:
from torch.utils.tensorboard import SummaryWriter

TypeError: ignored